In [2]:
# -*- coding: utf-8 -*-
"""
Created on Sat Feb  9 01:47:27 2019

@author: avaca
"""
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import median_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer

In [3]:
data = pd.read_csv('transformados04.csv')

In [4]:
data['exit_rate_page_views'] = data['GA_exit_rate']*data['GA_page_views']
data['views_precio_metro'] = data['GA_page_views'] * data['precio_metros']
target = data['TARGET']

In [5]:
Xt = data.drop(['TARGET'], axis = 1)

yt = data['TARGET']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(Xt, yt,
                                                    test_size = 0.1,
                                                    random_state = 69)

# Parámetros

In [7]:
# Número de árboles en el random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1100, num = 100)]

# Número de features a considerar en cada split
max_features = ['auto', 'sqrt']

# Máximo número de niveles en el árbol
max_depth = [None, 1, 2, 3]

# Número mínimo de samples para hacer un split en un nodo
min_samples_split = [2, 5, 10]

# Número mínimo de samples para cada hoja
min_samples_leaf = [1, 2, 4]

# Método de selección para cada árbol de entrenamiento
bootstrap = [True, False]

In [9]:
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [10]:
scorer = make_scorer(median_absolute_error)

In [18]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

# RandomForest

In [36]:
# rf = GridSearchCV(RandomForestRegressor(criterion="mse"),
#                   param_grid,
#                   cv = 5,
#                   scoring = scorer,
#                   n_jobs = -1,
#                   verbose = 1)

In [12]:
n_iteration = 20

In [13]:
rf = RandomizedSearchCV(RandomForestRegressor(criterion="mse"), param_grid, cv = 5,n_iter=n_iteration,
                        scoring = scorer, n_jobs = -1, verbose = 1)

In [14]:
rf.fit(X_train, y_train)

pred_rf = np.exp(rf.predict(X_test)) - 1
pred_rf = pred_rf.reshape(-1, 1)

df = pd.DataFrame(np.exp(y_test)-1 , index = y_test.index)
df.columns = ['TARGET']
df['pred_rf'] = pred_rf

median_absolute_error(df['TARGET'],df['pred_rf'])

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 18.2min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 41.3min finished


24.058408112399626

In [22]:
report(rf.cv_results_, n_top=20)

Model with rank: 1
Mean validation score: 0.549 (std: 0.014)
Parameters: {'n_estimators': 872, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 1, 'bootstrap': False}

Model with rank: 2
Mean validation score: 0.548 (std: 0.016)
Parameters: {'n_estimators': 763, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 1, 'bootstrap': True}

Model with rank: 3
Mean validation score: 0.548 (std: 0.016)
Parameters: {'n_estimators': 427, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 1, 'bootstrap': False}

Model with rank: 4
Mean validation score: 0.529 (std: 0.012)
Parameters: {'n_estimators': 918, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': None, 'bootstrap': False}

Model with rank: 5
Mean validation score: 0.525 (std: 0.016)
Parameters: {'n_estimators': 836, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 2, 'b

In [38]:
# filename = 'random_forest_gridsearchcv1.sav'
# pickle.dump(rf.best_estimator_, open(filename, 'wb'))

# ExtraTreesRegressor

In [39]:
etr = GridSearchCV(ExtraTreesRegressor(criterion="mse"),
                   param_grid,
                   cv = 5,
                   scoring = scorer,
                   n_jobs = -1,
                   verbose = 1)

In [40]:
etr.fit(X_train, y_train)

pred_etr = np.exp(etr.predict(X_test)) - 1
pred_etr = pred_etr.reshape(-1, 1)

df['pred_etr'] = pred_etr

median_absolute_error(df['TARGET'],df['pred_etr'])

Fitting 5 folds for each of 400 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 22.0min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 39.2min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 67.4min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 108.3min
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed: 125.8min finished


23.623642862137796

In [41]:
# filename = 'extra_trees_gridsearchcv1.sav'
# pickle.dump(etr.best_estimator_, open(filename, 'wb'))